In [1]:
#%pip install chatterbot-corpus -qq
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import NER_Attempt_02 as ner
import pickle

/home/dan/Desktop/Project_07/chatbot_env/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/dan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['next wednesday']


In [2]:
bot = ChatBot(
    'TravelJohnny',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3'
)


[nltk_data] Downloading package stopwords to /home/dan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Bring in George's conversations...


with open('frames_list.pkl', 'rb') as f:
    frames_list = pickle.load(f)


In [4]:
# Bring in my conversations...
with open('custom_list.pkl', 'rb') as f:
    custom_list = pickle.load(f)

#Massage for trainer
#print(custom_list)

custom_list_2 =[]

for s in custom_list:
    custom_list_2.append(s[0])
    
print(custom_list_2)

['Hi there', 'How can I help you?', 'Are there any flights from Toronto to San Diego', 'Yes, for which dates would you like?', 'June 3rd to next Wednesday', 'How much would like to spend?', 'Under 500', 'Great, Here are some options:', 'Good Morning', 'How can i help you?', 'I would like to see if I can travel from Seattle to Chicago tomorrow', 'When would you like to come back', 'In 5 weeks, but I am flexible on the day', 'What is your budget', 'Less than fifteen hundred dollars', 'Great, Here are some options:', 'Yo', 'How can I help you?', 'How much is it to go from San Diego to Los Angeles. I have to get there for a wedding.', 'When would you like to travel', 'March 1st to March 2nd', 'What is your budget', 'The cheapest you can find', 'Great, Here are some options:', 'Guten Tag', 'How can I help you?', 'I would like to fly to San Francisco on the 5th of December', 'What is your budget', 'Price is no object, and I would like first class if possible', 'Great, Here are some options:'

In [5]:
trainer = ListTrainer(bot)
trainer.train(frames_list)
trainer.train(custom_list_2)

List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [6]:
print('Hello, how can I help you?')

Locations_list = []
Dates_list = []
Money_list = []

cnt = 0
cnt_return = 0
cnt_money = 0 
no_bud = False
one_way = False

while True:
    try:
        
        #Parse the customer input
        cust_input = input()        
        travel_chat_preproc = ner.word_preproc([[cust_input]])
        travel_ner_out = ner.word_ner_all(travel_chat_preproc)   
        
        #See if we found any information
        if len(travel_ner_out['Locations'])>0:
            for a in travel_ner_out['Locations']:
                Locations_list.append(a) 
        
        if len(travel_ner_out['Dates'])>0:
            for b in travel_ner_out['Dates']:
                Dates_list.append(b) 
            
        if len(travel_ner_out['Money'])>0:
            for c in travel_ner_out['Money']:
                Money_list.append(c) 
        
        print(len(Locations_list),len(Dates_list),len(Money_list))
                
        
        if cnt<2:
            bot_input = bot.get_response(cust_input)
            print(bot_input)
        
        else:
            
            if len(Locations_list)<2:
                print('Where do you want to travel from?')
                
            elif len(Dates_list)<2 and one_way == False:
                
                if cnt_return == 0:
                    print('When do you want to return?')
                else:
                    one_way = True
                    continue
                    
                cnt_return = cnt_return + 1
                
            elif len(Money_list)<1 and no_bud == False:
                
                
                if cnt_money == 0:
                    print('How much do you want to spend?')
                else:
                    no_bud = True
                    continue
                    
                cnt_money = cnt_money + 1
                
        
            else:
                print('Great, here are some options:')
                #Use information above to search sky scanner
                
                #Convert Dates, Location (airport codes) for usage in Flight lookup
                
                #Location check/narrower
        
                #Find match score for locations
                
                
                for v in Locations_list:
                    print(ner.match_score_list(v))
                
                break
        
        cnt = cnt + 1
 
    except(KeyboardInterrupt, EOFError, SystemExit):
        break

Hello, how can I help you?
I'd like to go to Toronto next Tuesday
['next tuesday']
1 1 0
When would you like to travel
July 17th
1 2 0
What would the price be if I shortened my trip by one day?
800 Dollars
1 2 1
Where do you want to travel from?
New York
2 2 1
Great, here are some options:
         city  city_score                        state  state_score code
3511  Toronto         100           ON - International           32  YYZ
3292  Toronto         100   ON - Downtown Rail service           30  YBZ
3458  Toronto         100  ON - Toronto Island Airport           41  YTZ
3231  Toronto         100  ON - Guildwood Rail service           18  XLQ
          city  city_score              state  state_score code
2181  New York         100  NY - All airports           32  NYC
1472  New York         100       NY - Kennedy           30  JFK
1698  New York         100    NY - La Guardia           26  LGA


In [7]:
# If no second loaction, ask for start location

# If no second date, assume one way

# If no money, assume cheapest

# If no dates, ask for dates, if not look at today/oneway

# If more than 3 locations -> Multicity

# Chatterbot
# https://chatterbot.readthedocs.io/en/stable/training.html

#Sky Scanner
#https://rapidapi.com/skyscanner/api/skyscanner-flight-search/endpoints


# Integration with Slack

In [8]:
#bot.export_for_training('my_export.json')

In [9]:
import requests

L1 = 'YYZ'
L2 = 'JFK'
D1 = '2020-06-01'
D2 = '2020-07-01'

url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/US/USD/en-US/" + L1 + "-sky/" +  L2 + "-sky/" + D1

querystring = {"inboundpartialdate":D2}

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "e9ea65cb6bmsh7a9294203a09dfep163c42jsn05f9e4a2cceb"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"Routes":[],"Quotes":[{"QuoteId":1,"MinPrice":78.0,"Direct":true,"OutboundLeg":{"CarrierIds":[1907],"OriginId":96474,"DestinationId":60987,"DepartureDate":"2020-06-01T00:00:00"},"QuoteDateTime":"2020-04-04T11:58:00"},{"QuoteId":2,"MinPrice":78.0,"Direct":true,"OutboundLeg":{"CarrierIds":[838],"OriginId":96474,"DestinationId":60987,"DepartureDate":"2020-06-02T00:00:00"},"QuoteDateTime":"2020-04-07T11:33:00"},{"QuoteId":3,"MinPrice":80.0,"Direct":true,"OutboundLeg":{"CarrierIds":[954],"OriginId":96474,"DestinationId":60987,"DepartureDate":"2020-06-03T00:00:00"},"QuoteDateTime":"2020-04-07T11:44:00"},{"QuoteId":4,"MinPrice":80.0,"Direct":true,"OutboundLeg":{"CarrierIds":[954],"OriginId":96474,"DestinationId":60987,"DepartureDate":"2020-06-04T00:00:00"},"QuoteDateTime":"2020-04-07T11:51:00"},{"QuoteId":5,"MinPrice":80.0,"Direct":true,"OutboundLeg":{"CarrierIds":[1361],"OriginId":96474,"DestinationId":60987,"DepartureDate":"2020-06-05T00:00:00"},"QuoteDateTime":"2020-04-07T11:57:00"},{"Quo